In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls -lha1 ./drive/MyDrive/reduced_images_ILB.zip

-rw------- 1 root root 1011M Mar  9 18:30 ./drive/MyDrive/reduced_images_ILB.zip


In [4]:
!unzip ./drive/MyDrive/reduced_images_ILB.zip -d . > /dev/null

In [9]:
import pandas as pd
import numpy as np
from PIL import Image

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import transforms

In [10]:
PATH_TABULAR_DATA = "./drive/MyDrive/real_estate_price_prediction_processed_09042024.csv"

In [11]:
data_tab = pd.read_csv(PATH_TABULAR_DATA)

In [12]:
data_tab[["price_scaled"]] = MinMaxScaler().fit_transform(data_tab[["price"]])

In [13]:
data_tab.head()

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,price,price_scaled
0,35996577,0,43.643880,7.117183,8452,199,63.0,3.479524,3995.665362,205.385148,...,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,355000.0,0.145320
1,35811033,0,45.695757,4.895610,8234,3347,90.0,3.000000,3995.665362,223.000000,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190000.0,0.072778
2,35731841,12,47.966791,-1.220451,5235,1650,61.0,3.479524,370.000000,205.385148,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39000.0,0.006390
3,35886765,12,47.289292,-1.878805,1929,2050,142.0,3.479524,764.000000,217.000000,...,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,299000.0,0.120699
4,35781137,0,45.718992,4.844234,4476,3334,88.0,3.000000,3995.665362,205.385148,...,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,478000.0,0.199397


In [14]:
X_train_, X_test, y_train_, y_test= train_test_split(
    data_tab.drop(columns=["price", "price_scaled"]),
    data_tab["price_scaled"],
    test_size=0.20,
    random_state=123,
    shuffle=True
    )

In [15]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_,
    y_train_,
    test_size=0.20,
    random_state=123,
    shuffle=True
    )

In [16]:
X_reorganized = pd.concat(
    [
        pd.DataFrame(X_train).assign(group="train"),
        pd.DataFrame(X_val).assign(group="val"),
        pd.DataFrame(X_test).assign(group="test")
    ]
).reset_index(drop=True)


y_reorganized = pd.concat(
    [
        pd.DataFrame(y_train).assign(group="train"),
        pd.DataFrame(y_val).assign(group="val"),
        pd.DataFrame(y_test).assign(group="test")
    ]
).reset_index(drop=True)

In [17]:
X_reorganized.sample(5)

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,group
29871,35871777,0,43.557495,1.536234,7076,1422,124.0,3.000000,222.000000,55.000000,...,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,val
19070,35795849,12,49.431739,2.092756,658,2824,93.0,3.479524,527.000000,394.000000,...,0.0,15.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,train
35650,35823427,5,48.874547,2.356967,5606,3637,81.0,3.479524,3995.665362,205.385148,...,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
8906,35829559,5,45.746246,3.208665,2019,3085,143.0,3.479524,3995.665362,205.385148,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,train
5779,35824439,0,48.897615,2.255554,2000,4565,6846.0,4.000000,3995.665362,110.000000,...,0.0,8.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,train


In [11]:
#X_reorganized[["id_annonce", "group"]]

In [18]:
PATH_IMAGES_DATA = "./drive/MyDrive/real_estate_price_prediction_metadata_images_09042024.csv"

In [19]:
data_img = pd.read_csv(PATH_IMAGES_DATA)

In [20]:
data_img_agg = data_img.groupby('id_annonce')['long_path_image'].apply(list).reset_index(name='images')

In [21]:
data_img_groups = data_img_agg.merge(
    X_reorganized[["id_annonce", "group"]],
    how='left',
    on='id_annonce'
)

In [22]:
data_img_groups.sample(10)

,id_annonce,images,group
21114,35886623,[./reduced_images/train/ann_35886623/f3b0d22f5...,train
446,29699271,[./reduced_images/train/ann_29699271/ebd13e519...,train
20656,35882841,[./reduced_images/train/ann_35882841/acc1c2b2f...,train
14190,35841947,[./reduced_images/train/ann_35841947/91c5ca9ec...,train
3743,35761079,[./reduced_images/train/ann_35761079/2558581b3...,train
31796,36036675,[./reduced_images/train/ann_36036675/bd55406b2...,train
20075,35879631,[./reduced_images/train/ann_35879631/ad6589662...,val
23546,35928531,[./reduced_images/train/ann_35928531/d75a8bd16...,test
13613,35838173,[./reduced_images/train/ann_35838173/8fc69d9fc...,train
14314,35842431,[./reduced_images/train/ann_35842431/01643f2f6...,train


In [23]:
# Define the image transformation
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
        ),
])


In [255]:
class CustomDataset(Dataset):

    def __init__(
        self,
        data_features,
        data_target,
        data_img_groups,
        group='train',
        transform=None
        ) -> None:

        self.group = group

        self.data_features = data_features.query(
            f"group == '{self.group}'"
            )

        self.data_target = data_target.query(
            f"group == '{self.group}'"
            )

        self.data_img_groups = data_img_groups.query(
            f"group == '{self.group}'"
            )

        self.transform = transform

        self.model = models.resnet152(pretrained=True)
        self.model_layers = list(self.model.children())[:-1]
        self.model_feature_extract = torch.nn.Sequential(*self.model_layers)

        self.set_model_eval()

    def __len__(self):
        return len(self.data_features)

    def set_model_eval(self):
      """
      Set to evaluation model.
      """
      return self.model.eval()


    def get_model_embedding(self, img_tensor):

      with torch.no_grad():
          # Get the image features from the ResNet-152 model
          img_features = self.model_feature_extract(img_tensor)

      return img_features

    def load_image(self, path) -> Image.Image:

        return Image.open(path).convert("RGB")

    def __getitem__(self, index):

        images_path = self.data_img_groups.iloc[index]["images"]

        images_emmbeded = []

        for path in images_path:
          image = self.load_image(path)

          if self.transform:
            image = self.transform(image).unsqueeze(0)

            img_embedding = self.get_model_embedding(image)
            images_emmbeded.append(img_embedding.flatten())

        images_embbeding = torch.mean(
                          torch.Tensor(np.array(images_emmbeded)),
                          axis = 0
                          )

        features = self.data_features.drop(
            columns=["id_annonce","group"]
            ).iloc[index,:]

        #features = torch.from_numpy(features).float()

        target = self.data_target[["price_scaled"]].values[index]
        #target = np.array(target)
        #target = torch.from_numpy(target).float()

        sample = {}

        sample["images_embbeded"] = images_embbeding
        sample["features"] = torch.tensor(features.values, dtype=torch.float32)
        sample["target"] = torch.tensor(target, dtype=torch.float32).reshape(-1, 1)

        return sample

In [256]:
train_dataset = CustomDataset(
        data_features=X_reorganized.head(100),
        data_target=y_reorganized.head(100),
        data_img_groups=data_img_groups,
        group='train',
        transform=preprocess
    )

val_dataset = CustomDataset(
        data_features=X_reorganized,
        data_target=y_reorganized,
        data_img_groups=data_img_groups,
        group='val',
        transform=preprocess
)

test_dataset = CustomDataset(
        data_features=X_reorganized,
        data_target=y_reorganized,
        data_img_groups=data_img_groups,
        group='test',
        transform=preprocess
)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [257]:
BATCH_SIZE= 20 #1024

In [258]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [259]:
class TabularModel(nn.Module): #
  def __init__(self,input_dim, device):
    super(TabularModel,self).__init__()
    self.device = device
    self.fc1=nn.Linear(input_dim,64)
    self.fc2=nn.Linear(64,128)
    self.fc3=nn.Linear(128,256)
    self.fc4=nn.Linear(256,512)             # last layer
    self.DO = nn.Dropout(p=0.2)

  def forward(self,x):
    out=self.DO(torch.relu(self.fc1(x["features"].to(self.device))))
    out=self.DO(torch.relu(self.fc2(out)))
    out=self.DO(torch.relu(self.fc3(out)))
    out=self.fc4(out)
    return out


In [260]:
class ImageModel(nn.Module):
  def __init__(self, device):
    super(ImageModel,self).__init__()  # building connection with parent and child classes
    self.device = device
    self.fc1=nn.Linear(2048, 1024)
    self.DO = nn.Dropout(p=0.2)

  def forward(self,x):
    out = x["images_embbeded"].squeeze().to(self.device)
    out = self.DO(torch.tanh(self.fc1(out)))
    return out


In [261]:
class MultiModal(torch.nn.Module):
    def __init__(
        self, device
    ):
        super().__init__()
        self.device = device

        img_fe = ImageModel(device)
        self.img_fe = img_fe

        tab_fe = TabularModel(26, device)
        self.tab_fe = tab_fe

        self.fc_img1 = torch.nn.Linear(1024, 1024)
        self.fc_img2 = torch.nn.Linear(1024, 512)

        self.fc_tab1 = torch.nn.Linear(512, 256)
        self.fc_tab2 = torch.nn.Linear(256, 128)

        self.fc_multi1 = torch.nn.Linear(640, 512)
        self.fc_multi2 = torch.nn.Linear(512, 1)

    def forward(self, x):
        y_img = torch.relu(self.fc_img2(torch.tanh(self.fc_img1(self.img_fe(x)))))
        y_img = torch.flatten(y_img)

        y_tab = torch.relu(self.fc_tab2(torch.tanh(self.fc_tab1(self.tab_fe(x)))))
        y_tab = torch.flatten(y_tab)

        y = torch.relu(self.fc_multi1(torch.concat([y_img, y_tab], dim=-1)))

        return self.fc_multi2(y)

In [160]:
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

In [161]:
device

'cpu'

In [262]:
model = MultiModal(device=device)

In [263]:
model.to(device)

MultiModal(
  (img_fe): ImageModel(
    (fc1): Linear(in_features=2048, out_features=1024, bias=True)
    (DO): Dropout(p=0.2, inplace=False)
  )
  (tab_fe): TabularModel(
    (fc1): Linear(in_features=26, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=128, bias=True)
    (fc3): Linear(in_features=128, out_features=256, bias=True)
    (fc4): Linear(in_features=256, out_features=512, bias=True)
    (DO): Dropout(p=0.2, inplace=False)
  )
  (fc_img1): Linear(in_features=1024, out_features=1024, bias=True)
  (fc_img2): Linear(in_features=1024, out_features=512, bias=True)
  (fc_tab1): Linear(in_features=512, out_features=256, bias=True)
  (fc_tab2): Linear(in_features=256, out_features=128, bias=True)
  (fc_multi1): Linear(in_features=640, out_features=512, bias=True)
  (fc_multi2): Linear(in_features=512, out_features=1, bias=True)
)

In [241]:
class EarlyStopping:
    def __init__(self, patience, verbose=False, delta=0, path="checkpoint.pt"):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        return self.early_stop

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(
                f"Validation Loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model to {self.path}"
            )
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

    def load_checkpoint(self, model):
        """Loads the best model weights from the saved checkpoint."""
        model.load_state_dict(torch.load(self.path))

In [242]:
# Tunners
LR = 0.001
EPOCHS = 2

In [243]:
optimizer = optim.Adam(model.parameters(), lr=LR, amsgrad=True, weight_decay=LR * 0.1)

criterion = nn.L1Loss().to(device)

early_stopping = EarlyStopping(patience=int(EPOCHS * 0.5), verbose=True)

In [146]:
def train_eval(model, loader, optimizer, loss_func, is_training):
    loss_list = []
    stop = False
    model.train() if is_training else model.eval()

    with torch.set_grad_enabled(is_training):  # Enable gradients only in training
        for batch_idx, sample in tqdm(
            enumerate(loader),
            desc="Epoch_Train" if is_training else "Epoch_Val",
            total=len(loader),
        ):
            #sample = sample.to(device)
            target = sample['target'].to(device)
            if is_training:
                optimizer.zero_grad()  # Reset gradients
            output = model(
                {
                    'features': sample["features"],
                    'images_embbeded': sample['images_embbeded']
                 }
              )
            loss = loss_func(output, target.unsqueeze(0))

            if is_training:
                loss.backward()  # Compute gradients
                optimizer.step()  # Update weights

            loss_list.append(loss.item())

    if not is_training:
        stop = early_stopping(
            (sum(loss_list) / len(loss_list)), model
        )  # Early stopping decision

    return loss_list, stop

In [166]:
def calc_acc_loss(total_loss, loss):
    total_loss.append(sum(loss) / len(loss))

    #total_accuracy.append(100 * correct / len(loader.dataset))
    return total_loss

In [185]:
# Lists
total_loss_train = []
total_loss_val = []
total_accuracy_train = [1]
total_accuracy_val = [1]

In [244]:
torch.cuda.empty_cache()
try:
  torch.cuda.reset_peak_memory_stats()
except:
  pass

In [150]:
%%time
# Training and Evaluation loop
for epoch in range(EPOCHS):
    # Train Iteration
    loss_train, _ = train_eval(
        model, train_loader, optimizer, criterion, is_training=True
    )

    # Val Iteration
    loss_val = train_eval(
        model, val_loader, optimizer, criterion, is_training=False
    )

    # Calculate and record loss & accuracy for training and validation
    total_loss_train, total_accuracy_train = calc_acc_loss(
        total_loss_train,
        loss_train
    )

    # Calculate Loss and Accuracy
    total_loss_val, stop = calc_acc_loss(
        total_loss_val,
        loss_val
    )

    # Print metrics per epoch
    print(
        f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {total_loss_train[-1]:.4f}, Val Loss: {total_loss_val[-1]:.4f}"
    )

    if stop:
        print("Early Stopping")
        break

Epoch_Train:   0%|          | 0/5 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x12800 and 640x512)

In [264]:
test = next(iter(train_loader))

In [275]:
test.keys()

dict_keys(['images_embbeded', 'features', 'target'])

In [267]:
test['features'].size()

torch.Size([20, 26])

In [276]:
test['images_embbeded'].size()

torch.Size([20, 2048])

In [308]:
torch.tensor(
    pd.DataFrame(test['images_embbeded'].numpy()).values,
    dtype=torch.float32
    ).size()

torch.Size([20, 2048])

In [274]:
#test['target']

In [268]:
test['target'].size()

torch.Size([20, 1, 1])

In [297]:
#test['features']


array([[3.58014970e+07, 1.80000000e+01, 4.45013834e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.60480570e+07, 1.20000000e+01, 5.02890975e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.58973670e+07, 0.00000000e+00, 4.18854453e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.58453450e+07, 0.00000000e+00, 4.89913348e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.60272250e+07, 1.20000000e+01, 4.86342764e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.60130090e+07, 1.20000000e+01, 4.51924273e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [298]:
X = torch.tensor(X_reorganized.drop(columns=["group"]).values, dtype=torch.float32)

In [300]:
X.size()

torch.Size([37368, 27])

In [288]:
test["features"][:2]

tensor([[0.0000e+00, 4.8866e+01, 2.2840e+00, 5.6120e+03, 3.6480e+03, 1.0000e+02,
         3.4795e+00, 3.9957e+03, 2.0539e+02, 7.0000e+00, 3.1846e+01, 7.0000e+00,
         7.0000e+00, 5.0000e+00, 3.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
         1.2000e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0000e+00, 4.5191e+01, 5.7325e+00, 2.9740e+03, 1.7780e+03, 4.6000e+01,
         4.0000e+00, 3.9957e+03, 2.0539e+02, 7.0000e+00, 3.1846e+01, 7.0000e+00,
         0.0000e+00, 2.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         3.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00]])

In [316]:
test['images_embbeded'].unsqueeze(1)

tensor([[[0.6439, 0.3119, 1.2845,  ..., 0.2371, 0.2670, 0.0551]],

        [[0.3679, 0.4996, 0.2848,  ..., 0.3415, 0.1283, 0.5214]],

        [[1.4785, 0.2495, 0.3892,  ..., 0.8778, 0.2667, 0.1979]],

        ...,

        [[3.7764, 0.0607, 0.6373,  ..., 0.6391, 0.3320, 0.1383]],

        [[1.1862, 0.2691, 0.4586,  ..., 0.8491, 0.2238, 0.0904]],

        [[1.1385, 0.7494, 0.2776,  ..., 0.5345, 0.2826, 0.3217]]])

In [327]:
list_features = [test["features"][0], test["features"][1]]
list_images_embbeded= [test["images_embbeded"][0], test["images_embbeded"][1]]

In [328]:
torch.stack(list_features, dim=0)

tensor([[0.0000e+00, 4.8866e+01, 2.2840e+00, 5.6120e+03, 3.6480e+03, 1.0000e+02,
         3.4795e+00, 3.9957e+03, 2.0539e+02, 7.0000e+00, 3.1846e+01, 7.0000e+00,
         7.0000e+00, 5.0000e+00, 3.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
         1.2000e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0000e+00, 4.5191e+01, 5.7325e+00, 2.9740e+03, 1.7780e+03, 4.6000e+01,
         4.0000e+00, 3.9957e+03, 2.0539e+02, 7.0000e+00, 3.1846e+01, 7.0000e+00,
         0.0000e+00, 2.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         3.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00]])

In [329]:
torch.stack(list_features, dim=0).squeeze(-1)

tensor([[0.0000e+00, 4.8866e+01, 2.2840e+00, 5.6120e+03, 3.6480e+03, 1.0000e+02,
         3.4795e+00, 3.9957e+03, 2.0539e+02, 7.0000e+00, 3.1846e+01, 7.0000e+00,
         7.0000e+00, 5.0000e+00, 3.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
         1.2000e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0000e+00, 4.5191e+01, 5.7325e+00, 2.9740e+03, 1.7780e+03, 4.6000e+01,
         4.0000e+00, 3.9957e+03, 2.0539e+02, 7.0000e+00, 3.1846e+01, 7.0000e+00,
         0.0000e+00, 2.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         3.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00]])

In [ ]:
model(
    {
      'features': test["features"][0],
      'images_embbeded': test['images_embbeded'][0]
  }
)

In [291]:
output = model(
                {
                    'features': test["features"][:2].unsqueeze(1),
                    'images_embbeded': test['images_embbeded'][:2].unsqueeze(1),
                 }
              )

output

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1280 and 640x512)

In [282]:
outputs = []

for i in range(len(test["features"])):
  output = model(
                {
                    'features': test["features"][0],
                    'images_embbeded': test['images_embbeded'][0],
                 }
              )

  outputs.append(output)

outputs

[tensor([-0.0414], grad_fn=<ViewBackward0>),
 tensor([-0.0410], grad_fn=<ViewBackward0>),
 tensor([-0.0031], grad_fn=<ViewBackward0>),
 tensor([-0.0442], grad_fn=<ViewBackward0>),
 tensor([-0.0570], grad_fn=<ViewBackward0>),
 tensor([0.0073], grad_fn=<ViewBackward0>),
 tensor([-0.0063], grad_fn=<ViewBackward0>),
 tensor([-0.0616], grad_fn=<ViewBackward0>),
 tensor([0.0110], grad_fn=<ViewBackward0>),
 tensor([-0.0793], grad_fn=<ViewBackward0>),
 tensor([-0.0184], grad_fn=<ViewBackward0>),
 tensor([-0.0152], grad_fn=<ViewBackward0>),
 tensor([-0.0048], grad_fn=<ViewBackward0>),
 tensor([-0.0383], grad_fn=<ViewBackward0>),
 tensor([0.0163], grad_fn=<ViewBackward0>),
 tensor([-0.0554], grad_fn=<ViewBackward0>),
 tensor([0.0308], grad_fn=<ViewBackward0>),
 tensor([-0.0028], grad_fn=<ViewBackward0>),
 tensor([0.0008], grad_fn=<ViewBackward0>),
 tensor([-0.0115], grad_fn=<ViewBackward0>)]

In [121]:
outputs

[tensor([-0.0337], grad_fn=<ViewBackward0>)]

In [208]:
output = model(
                {
                    'features': test["features"],
                    'images_embbeded': test['images_embbeded']
                 }
              )

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x12800 and 640x512)

In [210]:
12800/20

640.0

## Texting

In [ ]:
try:
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
except:
  pass

In [ ]:
model_pred = MultiModal(device=device)
model_pred.load_state_dict(torch.load("checkpoint.pt"))
model_pred.to(device)

In [ ]:
def test(model, loader):
    model.eval()
    total_loss_test = []

    pred = torch.Tensor().to(device)

    with torch.no_grad():
        for batch_idx, sample in tqdm(
            enumerate(loader), desc="Test Epoch", total=len(loader)
        ):
            target = sample["feature"].to(device)
            output = model(sample)

            loss = criterion(output, target)
            total_loss_test.append(loss.item())

            pred = torch.cat((pred, test_pred), 0)

        return total_loss_test